# Ever Given

<img align="right" src="https://anitagraser.github.io/movingpandas/assets/img/movingpandas.png">

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/anitagraser/movingpandas-examples/main?filepath=3-tech-demos/ever-given.ipynb)

This notebook presents an analysis of the vessel situation following the grounding of Ever Given in the Suez Canal.


The dataset used covers the time span between 2021-03-20 00:00 and 2021-03-24 12:52 UTC.
 
This data has generously been provided by VesselsValue.


<img align="left" src="images/Ever-Given-Suez.jpg">


In [ ]:
%matplotlib inline

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import pandas as pd
import geopandas as gpd
import movingpandas as mpd
from geopandas import GeoDataFrame, read_file
from shapely.geometry import Point, LineString, Polygon
from datetime import datetime, timedelta

from holoviews import opts, dim
from holoviews.selection import link_selections

import warnings
warnings.simplefilter("ignore")

In [ ]:
EVERID = 235
FSIZE = 500

In [ ]:
df = pd.read_csv('E:/Geodata/VesselsValue/boat-positions.csv')
df['t'] = pd.to_datetime(df['ais_pos_timestamp'], format='%d/%m/%Y %H:%M')
df = df.set_index('t').tz_localize(None)
df

In [ ]:
gdf = gpd.GeoDataFrame(df.drop(['longitude', 'latitude', 'ais_pos_timestamp'], axis=1), 
                       crs='epsg:4326', geometry=[Point(xy) for xy in zip(df.longitude, df.latitude)])
gdf

In [ ]:
gdf.hvplot(geo=True, tiles='OSM', frame_width=FSIZE, frame_height=FSIZE)

In [ ]:
trajs = mpd.TrajectoryCollection(gdf, 'ID')

In [ ]:
evergiven = trajs.get_trajectory(EVERID)
evergiven.hvplot(line_width=7, frame_width=FSIZE, frame_height=FSIZE)

In [ ]:
stop_pts = mpd.TrajectoryStopDetector(trajs).get_stop_points(min_duration=timedelta(hours=3), max_diameter=1000)

In [ ]:
stop_pts['ID'] = stop_pts.index
stop_pts['ID'] = stop_pts['ID'].apply(lambda row: int(row.split('_')[0]))
stop_pts['duration_h'] = (stop_pts['end_time']-stop_pts['start_time']).dt.total_seconds() / 3600
stop_pts

## Ever Given ran aground around 7:40 local time (5:40 UTC) on 23rd March 2021

In [ ]:
stop_pts[stop_pts['ID']==EVERID]

In [ ]:
plot = evergiven.hvplot(line_width=5, color='red', frame_width=FSIZE, frame_height=FSIZE, alpha=0.5)
plot = plot * stop_pts.hvplot(geo=True, hover_cols=['start_time'], size=20) 
plot = plot * stop_pts[stop_pts['ID']==EVERID].hvplot(geo=True, hover_cols=['start_time'], size=dim('duration_h')/2, color='red', 
                                                      title='Trajectory & stop location of Ever Given and stops of other vessels')
plot2 = pd.DataFrame(stop_pts).hvplot.scatter(title='Stop start & duration (in hours)', x='start_time', y='duration_h', frame_width=FSIZE, frame_height=FSIZE) 
plot2 = plot2 * pd.DataFrame(stop_pts[stop_pts['ID']==EVERID]).hvplot.scatter(x='start_time', y='duration_h', color='red', size=200)
link_selections(plot + plot2)#.cols(1)

*Data generously provided by VesselsValue.*

In [ ]:
from datashader.utils import lnglat_to_meters
from holoviews.element import tiles

df = stop_pts.copy()
df['lon'] = df.geometry.x
df['lat'] = df.geometry.y
df = pd.DataFrame(df)
df.loc[:, 'x'], df.loc[:, 'y'] = lnglat_to_meters(df.lon, df.lat)

plot = tiles.OSM() * df.hvplot.scatter(x='x', y='y', frame_width=FSIZE, frame_height=FSIZE, title='Trajectory & stop location of Ever Given and stops of other vessels')
plot2 = df.hvplot.scatter(title='Stop start & duration (in hours)', x='start_time', y='duration_h', frame_width=FSIZE, frame_height=FSIZE)

link_selections(plot + plot2)#.cols(1)